# Laboratorio 6
## Hunting with Data Science

---

Credo por:

- Juan Fernando De Leon Quezada
- Cristopher Barrios

In [33]:
# Librerias

import json
import pandas as pd
from flare.data_science.features import domain_tld_extract
from joblib import dump, load
from urllib.parse import urlparse, urlencode
from flare.data_science.features import dga_classifier

### Parte 1
---

1. Cargue la información del archivo large_even.json en una lista, muestre la cantidad de registros total (deben ser 746, 909).

In [34]:
# Load data

json_data = open("large_eve_json/large_eve.json")

data_array = []
for i in json_data.readlines():
    if i:
        data_array.append(json.loads(i))

json_data.close()

print("Total Data: ", len(data_array))

Total Data:  746909


2. Debido a que estamos buscando dominios DGA, del total de registros, solamente estamos interesados en los registros DNS. Cargue únicamente aquellos registros en cuya llave se encuentra “DNS”.

* Muestre la nueva cantidad de registros filtrados.

In [35]:
# DNS data

dns = []

for i in data_array:
    if "dns" in i.keys():
        dns.append(i)

print("Total DNS data: ", len(dns))

Total DNS data:  15749


* Muestre la información de 2 registros cualesquiera.

In [36]:
print("{}\n{}".format(dns[0], dns[1]))

{'timestamp': '2017-07-22T17:33:16.661646-0500', 'flow_id': 1327836194150542, 'pcap_cnt': 22269, 'event_type': 'dns', 'vlan': 110, 'src_ip': '2001:0dbb:0c18:0011:0260:6eff:fe30:0863', 'src_port': 59680, 'dest_ip': '2001:0500:0001:0000:0000:0000:803f:0235', 'dest_port': 53, 'proto': 'UDP', 'dns': {'type': 'query', 'id': 15529, 'rrname': 'api.wunderground.com', 'rrtype': 'A', 'tx_id': 0}}
{'timestamp': '2017-07-22T17:33:24.990320-0500', 'flow_id': 2022925111925872, 'pcap_cnt': 54352, 'event_type': 'dns', 'vlan': 110, 'src_ip': '2001:0dbb:0c18:0011:0260:6eff:fe30:0863', 'src_port': 38051, 'dest_ip': '2001:0500:0003:0000:0000:0000:0000:0042', 'dest_port': 53, 'proto': 'UDP', 'dns': {'type': 'query', 'id': 58278, 'rrname': 'stork79.dropbox.com', 'rrtype': 'A', 'tx_id': 0}}


* Debido a que la data consiste en json anidados, utilice la característica json_normalize para normalizar la información y asignarla en un dataframe.

In [37]:
# JSON Normalize

df = pd.json_normalize(dns)
df.head()

,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,dns.type,dns.id,dns.rrname,dns.rrtype,dns.tx_id,dns.rcode,dns.ttl,dns.rdata
0,2017-07-22T17:33:16.661646-0500,1327836194150542,22269,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680,2001:0500:0001:0000:0000:0000:803f:0235,53,UDP,query,15529,api.wunderground.com,A,0.0,NaN,NaN,NaN
1,2017-07-22T17:33:24.990320-0500,2022925111925872,54352,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,38051,2001:0500:0003:0000:0000:0000:0000:0042,53,UDP,query,58278,stork79.dropbox.com,A,0.0,NaN,NaN,NaN
2,2017-07-22T17:33:27.379891-0500,578544790391795,54519,dns,150,192.168.205.170,31393,192.168.207.4,53,UDP,query,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,A,0.0,NaN,NaN,NaN
3,2017-07-22T17:33:27.380146-0500,578544790391795,54520,dns,150,192.168.207.4,53,192.168.205.170,31393,UDP,answer,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,NaN,NaN,NXDOMAIN,NaN,NaN
4,2017-07-22T17:33:27.380146-0500,578544790391795,54520,dns,150,192.168.207.4,53,192.168.205.170,31393,UDP,answer,54724,<root>,SOA,NaN,NXDOMAIN,20864.0,NaN


* Como estamos buscando dominios DGA, debemos filtrar los registros DNS para aquellos registros tipo A (son aquellos que mantienen una dirección IP asociada a un dominio), se debió bajar la cantidad a 2849 registros.

In [38]:
# DGA registers

a_reg = df[df['dns.rrtype'] == 'A']
a_reg['dns.rrtype'].value_counts()

A    2849
Name: dns.rrtype, dtype: int64

* Filtre los dominios únicos.

In [39]:
# Unique

unique = a_reg.drop_duplicates(subset=['dns.rrname'])
unique.head()

,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,dns.type,dns.id,dns.rrname,dns.rrtype,dns.tx_id,dns.rcode,dns.ttl,dns.rdata
0,2017-07-22T17:33:16.661646-0500,1327836194150542,22269,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680,2001:0500:0001:0000:0000:0000:803f:0235,53,UDP,query,15529,api.wunderground.com,A,0.0,NaN,NaN,NaN
1,2017-07-22T17:33:24.990320-0500,2022925111925872,54352,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,38051,2001:0500:0003:0000:0000:0000:0000:0042,53,UDP,query,58278,stork79.dropbox.com,A,0.0,NaN,NaN,NaN
2,2017-07-22T17:33:27.379891-0500,578544790391795,54519,dns,150,192.168.205.170,31393,192.168.207.4,53,UDP,query,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,A,0.0,NaN,NaN,NaN
6,2017-07-22T17:33:38.537426-0500,2167545251640146,55687,dns,180,192.168.198.62,35092,192.168.207.4,53,UDP,query,7425,safebrowsing.clients.google.com.home,A,0.0,NaN,NaN,NaN
9,2017-07-22T17:33:45.672579-0500,541745511744323,56307,dns,120,192.168.202.172,55580,192.168.207.4,53,UDP,query,12854,fxfeeds.mozilla.com,A,0.0,NaN,NaN,NaN


In [40]:
print(len(unique))

177


* Del dataframe de dominios únicos de tipo A, obtenga el TLD (top level domain) utilizando Flare como una columna nueva.

In [41]:
# Domain TLD Extract

unique['tld_extract_domain'] = unique['dns.rrname'].apply(domain_tld_extract)
unique.head()

C:\Users\jfdel\AppData\Local\Temp\ipykernel_7212\2081463357.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique['tld_extract_domain'] = unique['dns.rrname'].apply(domain_tld_extract)


,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,dns.type,dns.id,dns.rrname,dns.rrtype,dns.tx_id,dns.rcode,dns.ttl,dns.rdata,tld_extract_domain
0,2017-07-22T17:33:16.661646-0500,1327836194150542,22269,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680,2001:0500:0001:0000:0000:0000:803f:0235,53,UDP,query,15529,api.wunderground.com,A,0.0,NaN,NaN,NaN,wunderground.com
1,2017-07-22T17:33:24.990320-0500,2022925111925872,54352,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,38051,2001:0500:0003:0000:0000:0000:0000:0042,53,UDP,query,58278,stork79.dropbox.com,A,0.0,NaN,NaN,NaN,dropbox.com
2,2017-07-22T17:33:27.379891-0500,578544790391795,54519,dns,150,192.168.205.170,31393,192.168.207.4,53,UDP,query,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,A,0.0,NaN,NaN,NaN,aoltw.net
6,2017-07-22T17:33:38.537426-0500,2167545251640146,55687,dns,180,192.168.198.62,35092,192.168.207.4,53,UDP,query,7425,safebrowsing.clients.google.com.home,A,0.0,NaN,NaN,NaN,safebrowsing.clients.google.com.home
9,2017-07-22T17:33:45.672579-0500,541745511744323,56307,dns,120,192.168.202.172,55580,192.168.207.4,53,UDP,query,12854,fxfeeds.mozilla.com,A,0.0,NaN,NaN,NaN,mozilla.com


### Parte 2
---

* Utilice su clasificador entrenado en el laboratorio 1, y clasifique los dominios del dataframe de la parte 1. Filtre aquellos claisificados como DGA y muéstrelos.

In [42]:
# Load CLF

clf = load('clf.joblib')

c:\Users\jfdel\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.0.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [43]:
unique = unique.reset_index()

In [44]:
import re

#Calculate URL Length
unique['url_length'] = unique['dns.rrname'].str.len()

#Calculate Hostname Length
unique['hostname_length'] = unique['tld_extract_domain'].str.len()

for index, row in unique.iterrows():
#     print(index, row)
#     print(unique.loc[unique.index[index], 'special_chars'])
    unique.loc[unique.index[index], 'special_chars'] = new = len(re.findall('[.@?&j=_~%/*:,;$]', row['dns.rrname']))

unique['url_digits'] = unique['dns.rrname'].str.count('[0-9]')

unique['hostname_digits'] = unique['tld_extract_domain'].str.count('[0-9]')

unique.head()

,index,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,...,dns.tx_id,dns.rcode,dns.ttl,dns.rdata,tld_extract_domain,url_length,hostname_length,special_chars,url_digits,hostname_digits
0,0,2017-07-22T17:33:16.661646-0500,1327836194150542,22269,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680,2001:0500:0001:0000:0000:0000:803f:0235,53,...,0.0,NaN,NaN,NaN,wunderground.com,20,16,2.0,0,0
1,1,2017-07-22T17:33:24.990320-0500,2022925111925872,54352,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,38051,2001:0500:0003:0000:0000:0000:0000:0042,53,...,0.0,NaN,NaN,NaN,dropbox.com,19,11,2.0,2,0
2,2,2017-07-22T17:33:27.379891-0500,578544790391795,54519,dns,150,192.168.205.170,31393,192.168.207.4,53,...,0.0,NaN,NaN,NaN,aoltw.net,42,9,7.0,1,0
3,6,2017-07-22T17:33:38.537426-0500,2167545251640146,55687,dns,180,192.168.198.62,35092,192.168.207.4,53,...,0.0,NaN,NaN,NaN,safebrowsing.clients.google.com.home,36,36,4.0,0,0
4,9,2017-07-22T17:33:45.672579-0500,541745511744323,56307,dns,120,192.168.202.172,55580,192.168.207.4,53,...,0.0,NaN,NaN,NaN,mozilla.com,19,11,2.0,0,0


In [45]:
#  ['hostname', 'status', 'url_length', 'hostname_length', 'https', 'url_digits', 'hostname_digits', 'phish_hints', 'special_chars', 'url_digits', 'hostname_digits', 'domain_in_brand', 'brand_in_subdomain', 'google_index']

final_df = unique[['url_length', 'hostname_length', 'url_digits', 'hostname_digits', 'special_chars', ]]

final_df.head()

,url_length,hostname_length,url_digits,hostname_digits,special_chars
0,20,16,0,0,2.0
1,19,11,2,0,2.0
2,42,9,1,0,7.0
3,36,36,0,0,4.0
4,19,11,0,0,2.0


In [46]:
# Predict

pred = clf.predict(final_df)

In [47]:
print(pred)

[1 0 0 1 1 1 0 0 0 0 0 0 0 1 1 0 0 0 1 1 1 0 1 0 1 1 0 1 1 1 1 1 0 1 1 1 1
 0 1 1 1 1 0 0 1 0 1 0 1 1 0 0 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1
 1 1 1 1 1 1 1 0 1 1 1 1 0 1 0 1 1 0 1 0 1 0 0 1 1 1 1 0 0 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 0 0 1 0 1 1 0]


In [48]:
dga_registers = []
for i in pred:
    if i == 0:
        dga_registers.append(False)
    else:
        dga_registers.append(True)

In [49]:
unique[dga_registers]['tld_extract_domain']

0                          wunderground.com
3      safebrowsing.clients.google.com.home
4                               mozilla.com
5                            metasploit.com
13                                wpad.home
                       ...                 
168                    192.168.21-28.0.home
169                         192.168.22.201:
172              ntp.ubuntu.com.localdomain
174                            verisign.com
175          192.168.22.201:.stayonline.net
Name: tld_extract_domain, Length: 135, dtype: object

* Utilice el clasificador de DGA que incluye Flare, y clasifique los dominios (agregue a su dataframe la columna de clase). Filtre aquellos considerados como DGA y muéstrelos. ¿Son los mismos dominios que los de su modelo?

---- No todos pero si la gran mayoria son los mismos dominios que nuestro modelo

In [50]:
### DGA por Flare
## Da error
# from data_science.features import dga_classifier
# dga_c = dga_classifier()

In [51]:
unique['is_legit'] = unique['tld_extract_domain']
unique.head()

,index,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,...,dns.rcode,dns.ttl,dns.rdata,tld_extract_domain,url_length,hostname_length,special_chars,url_digits,hostname_digits,is_legit
0,0,2017-07-22T17:33:16.661646-0500,1327836194150542,22269,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680,2001:0500:0001:0000:0000:0000:803f:0235,53,...,NaN,NaN,NaN,wunderground.com,20,16,2.0,0,0,wunderground.com
1,1,2017-07-22T17:33:24.990320-0500,2022925111925872,54352,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,38051,2001:0500:0003:0000:0000:0000:0000:0042,53,...,NaN,NaN,NaN,dropbox.com,19,11,2.0,2,0,dropbox.com
2,2,2017-07-22T17:33:27.379891-0500,578544790391795,54519,dns,150,192.168.205.170,31393,192.168.207.4,53,...,NaN,NaN,NaN,aoltw.net,42,9,7.0,1,0,aoltw.net
3,6,2017-07-22T17:33:38.537426-0500,2167545251640146,55687,dns,180,192.168.198.62,35092,192.168.207.4,53,...,NaN,NaN,NaN,safebrowsing.clients.google.com.home,36,36,4.0,0,0,safebrowsing.clients.google.com.home
4,9,2017-07-22T17:33:45.672579-0500,541745511744323,56307,dns,120,192.168.202.172,55580,192.168.207.4,53,...,NaN,NaN,NaN,mozilla.com,19,11,2.0,0,0,mozilla.com


In [52]:
unique['is_legit'].value_counts()

aoltw.net                              15
aol.com                                11
google.com                             10
microsoft.com                           6
stayonline.net                          5
                                       ..
flickr.com                              1
windowsupdate.com                       1
"192.168.206.56"                        1
safebrowsing.clients.google.com.lan     1
real.com                                1
Name: is_legit, Length: 113, dtype: int64

In [53]:
unique[unique['is_legit'] == 'dga']['tld_extract_domain']

Series([], Name: tld_extract_domain, dtype: object)

### Parte 3
---

- Ahora ya tenemos un listado de dominios reducido y considerado como sospechoso, por lo 
que debemos aplicar dominio experto para encontrar los verdaderos registros malicioso. 
Utilizando la lista de Cisco Umbrella incluido en Flare, indique si los dominios encontrados 
por el clasificador DGA de Flare se encuentran en este top 1000000. 

In [54]:
from flare.tools.umbrella import Umbrella
cisVal = Umbrella(limit = 1000000)

inCi = []
nonCi = []
for i in unique[unique['is_legit'] == 'dga']['tld_extract_domain']:
    umbrellaAns = cisVal.domain_tld_in_umbrella(i)
    if umbrellaAns == True:
        inCi.append(i)
    else:
        nonCi.append(i)

[+] Updated Umbrella Top 1 Million list...


In [55]:

print(inCi)
print(nonCi)

[]
[]


- Si son considerados por el clasificador como DGA, y no están en el top de Cisco Umbrella es 
muy probable que sean maliciosos. Para confirmar esto podemos buscar la fecha de creación 
del dominio. Cree una función qué en base al dominio, devuelva la fecha de creación de este 
(utilice la librería whois para esto). 

In [56]:
import whois

def fecha_creacion_dom(dom):
    whoisF = whois.whois(dom)
    return whoisF.fecha_creacion_dom



- Muestre la fecha de creación para cada uno de los dominios clasificados como DGA. ¿Cuáles 
son los dominios que podemos confirmar como sospechosos?

--- En este caso los mas sospechosos deberian de ser los que ya no aparecen en umbrella, es dificil tambien de ver ya que no tiene fecha en la que fueron creados, ademas que varios de ellos aparecen sin fecha

In [57]:
fechaCreacion = []
sinFecha = []

for i in unique[unique['is_legit'] == 'dga']['tld_extract_domain']:
    try:
        time = fecha_creacion_dom(i)
        if time != None:
            fechaCreacion.append(i)
        else:
            sinFecha.append(i)
    except:
        print(i, "No!")


In [58]:

print(fechaCreacion)
print(sinFecha)

[]
[]
